In [1]:
import pandas as pd
import numpy as np
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1%}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
# add trip type to trip tables
def prep_trip_df(trip):
    """Mode shares will represent shares across Drive Alone, Shared Ride, 
       public transit, and walk and bike trips only. 
    """
    df_trip = trip.copy()
    df_trip.rename(columns={'mode':'Mode'}, inplace=True)
    df_trip['Trip Type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'Trip Type'] = 'Work'

    # add all tripa
    df = df_trip.copy()
    df['Trip Type'] = 'All Trips'
    df_trip = pd.concat([df_trip.sort_values('Trip Type'), df], ignore_index=True)

    # Exclude TNC, School Bus, and Other modes
    df_trip = df_trip[~df_trip['Mode'].isin(['TNC', 'School Bus', 'Other'])]

    # Aggregate modes
    df_trip['Mode'] = df_trip['Mode'].replace({
        'SOV': 'Drove Alone',
        'HOV2': 'Shared Ride',
        'HOV3+': 'Shared Ride',
        'Transit': 'Transit',
        'Walk': 'Walk',
        'Bike': 'Bike',
    })

    return df_trip

In [4]:
person = pd.read_csv(output_path / 'agg/dash/person_geog.csv')

# counties
trip_county = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_county.csv'))
# remove "Outside Region"
trip_county = trip_county[trip_county['hh_county'] != 'Outside Region']
# regional geographies
trip_rg = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_rg.csv'))
# centers
trip_rgc = pd.read_csv(output_path / 'agg/dash/mode_share_rgc.csv')
# add all RGCs
df = trip_rgc[trip_rgc['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
df['hh_rgc'] = 'All RGCs'
trip_rgc = pd.concat([df, trip_rgc])
trip_rgc = prep_trip_df(trip_rgc)

In [5]:
# equity geographies
equity_geogs = summary_config['hh_equity_geogs']
trip_equity_geog = pd.DataFrame()
for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])


## regional mode share

In [6]:
df1 = trip_county.groupby(['Trip Type','Mode'],as_index=False)['trexpfac'].sum()
df2 = trip_county.groupby(['Trip Type'],as_index=False)['trexpfac'].sum()
df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

df = df1.merge(df2, on='Trip Type')
df['mode_share'] = df['trexpfac'] / df['total_trips']

df.pivot_table(columns='Mode', index='Trip Type', values='mode_share')

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
Trip Type,,,,,
All Trips,1.5%,45.4%,36.1%,2.2%,14.8%
Non-Work,1.5%,41.2%,39.4%,2.2%,15.7%
Work,1.4%,72.0%,15.3%,2.3%,8.9%


## trip mode share by home location

In [7]:
def calc_mode_share(df_trip, geog):
    
    # num trips by mode
    df1 = df_trip.groupby([geog, 'Trip Type', 'Mode'],as_index=False)['trexpfac'].sum()
    # num trips
    df2 = df_trip.groupby([geog, 'Trip Type'],as_index=False)['trexpfac'].sum()
    df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

    df = df1.merge(df2, on=[geog, 'Trip Type'])
    # mode share
    df['mode_share'] = df['trexpfac'] / df['total_trips']

    # mode share by trip type
    df_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    # num trips by trip type
    trip_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')

    return df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work


df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work = calc_mode_share(trip_county,'hh_county')

In [8]:
df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,1.7%,44.2%,34.0%,3.3%,16.8%
Kitsap,1.2%,46.1%,37.0%,1.3%,14.4%
Pierce,1.4%,46.0%,39.4%,0.8%,12.4%
Snohomish,1.3%,47.8%,38.4%,0.9%,11.7%


In [9]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"146,066","3,833,573","2,949,490","286,330","1,457,699"
Kitsap,"12,293","457,720","367,885","12,567","143,150"
Pierce,"45,608","1,482,008","1,270,791","25,024","399,150"
Snohomish,"40,382","1,478,822","1,187,229","26,349","361,026"


In [10]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,1.7%,40.1%,37.2%,3.4%,17.7%
Kitsap,1.3%,42.5%,39.9%,0.9%,15.4%
Pierce,1.5%,41.9%,42.6%,0.7%,13.3%
Snohomish,1.4%,43.4%,41.7%,0.9%,12.7%


In [11]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"124,820","2,980,805","2,766,722","249,697","1,315,031"
Kitsap,"11,194","373,321","350,510","7,926","135,021"
Pierce,"41,431","1,188,447","1,208,508","20,115","378,485"
Snohomish,"36,515","1,166,860","1,121,455","23,657","340,986"


In [12]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,1.7%,69.0%,14.8%,3.0%,11.5%
Kitsap,1.0%,73.0%,15.0%,4.0%,7.0%
Pierce,1.1%,76.1%,16.2%,1.3%,5.4%
Snohomish,1.0%,77.2%,16.3%,0.7%,5.0%


In [13]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"21,246","852,768","182,768","36,633","142,668"
Kitsap,"1,099","84,399","17,375","4,641","8,129"
Pierce,"4,177","293,561","62,283","4,909","20,665"
Snohomish,"3,867","311,962","65,774","2,692","20,040"


In [14]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rgc, 'hh_rgc')

df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.1%,29.7%,16.3%,6.6%,44.3%
Auburn,2.5%,38.5%,29.6%,3.4%,26.0%
Bellevue,2.6%,32.1%,16.9%,3.7%,44.8%
Bothell Canyon Park,1.3%,51.0%,32.9%,0.6%,14.1%
Bremerton,2.4%,36.9%,21.8%,2.3%,36.6%
Burien,1.9%,44.4%,27.3%,3.2%,23.1%
Everett,1.9%,38.7%,20.6%,2.0%,36.7%
Federal Way,1.9%,36.6%,36.4%,1.9%,23.2%
Greater Downtown Kirkland,1.9%,47.4%,26.0%,3.8%,20.9%


In [15]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"33,703","324,897","177,977","72,343","484,583"
Auburn,216,"3,288","2,526",288,"2,215"
Bellevue,"1,712","21,226","11,163","2,415","29,612"
Bothell Canyon Park,34,"1,307",843,16,361
Bremerton,305,"4,705","2,773",293,"4,664"
Burien,288,"6,643","4,095",485,"3,465"
Everett,509,"10,148","5,411",536,"9,637"
Federal Way,44,844,839,44,534
Greater Downtown Kirkland,599,"15,092","8,286","1,220","6,641"


In [16]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.0%,26.7%,17.9%,7.0%,45.3%
Auburn,2.5%,35.5%,31.4%,2.9%,27.7%
Bellevue,2.5%,28.6%,18.5%,4.0%,46.5%
Bothell Canyon Park,1.4%,46.7%,36.1%,0.5%,15.3%
Bremerton,2.3%,33.2%,23.6%,1.6%,39.3%
Burien,1.9%,39.6%,29.2%,3.4%,26.0%
Everett,1.9%,34.2%,22.2%,2.2%,39.4%
Federal Way,1.9%,33.1%,38.5%,2.0%,24.6%
Greater Downtown Kirkland,1.8%,43.0%,28.4%,3.9%,22.9%


In [17]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"27,449","242,761","163,202","63,708","412,122"
Auburn,188,"2,682","2,374",219,"2,093"
Bellevue,"1,400","15,823","10,230","2,204","25,752"
Bothell Canyon Park,30,"1,010",781,10,332
Bremerton,251,"3,683","2,617",178,"4,361"
Burien,242,"5,046","3,719",436,"3,315"
Everett,431,"7,758","5,038",506,"8,935"
Federal Way,39,676,786,41,502
Greater Downtown Kirkland,503,"11,702","7,734","1,049","6,218"


In [18]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.4%,44.6%,8.0%,4.7%,39.3%
Auburn,2.9%,62.0%,15.6%,7.1%,12.5%
Bellevue,2.9%,50.4%,8.7%,2.0%,36.0%
Bothell Canyon Park,1.0%,74.6%,15.6%,1.5%,7.3%
Bremerton,3.3%,61.9%,9.5%,7.0%,18.4%
Burien,2.1%,72.0%,17.0%,2.2%,6.8%
Everett,2.2%,66.9%,10.4%,0.8%,19.6%
Federal Way,1.9%,64.4%,20.3%,1.1%,12.3%
Greater Downtown Kirkland,2.1%,73.2%,11.9%,3.7%,9.1%


In [19]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"6,254","82,136","14,775","8,635","72,461"
Auburn,28,606,152,69,122
Bellevue,312,"5,403",933,211,"3,860"
Bothell Canyon Park,4,297,62,6,29
Bremerton,54,"1,022",156,115,303
Burien,46,"1,597",376,49,150
Everett,78,"2,390",373,30,702
Federal Way,5,168,53,3,32
Greater Downtown Kirkland,96,"3,390",552,171,423


In [20]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rg, 'hh_rg_proposed')

df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,1.2%,46.0%,40.1%,0.6%,12.1%
Core Cities,1.4%,46.6%,37.8%,1.9%,12.4%
High Capacity Transit Communities,1.3%,47.1%,38.7%,1.4%,11.5%
Metropolitan Cities,2.1%,40.9%,28.9%,4.4%,23.7%
Rural Areas,0.9%,50.8%,41.8%,0.4%,6.1%
Urban Unincorporated Areas,1.2%,45.9%,42.8%,0.5%,9.5%


In [21]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"16,673","639,177","556,893","8,085","167,444"
Core Cities,"53,052","1,717,820","1,394,323","68,385","456,336"
High Capacity Transit Communities,"46,243","1,641,274","1,347,153","48,003","401,668"
Metropolitan Cities,"103,550","1,997,892","1,412,681","215,468","1,158,972"
Rural Areas,"17,074","966,426","794,275","7,378","116,489"
Urban Unincorporated Areas,"7,757","289,544","270,082","2,951","60,116"


In [22]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,1.3%,41.6%,43.4%,0.5%,13.1%
Core Cities,1.5%,42.3%,41.1%,1.8%,13.4%
High Capacity Transit Communities,1.4%,42.8%,42.0%,1.3%,12.5%
Metropolitan Cities,2.1%,36.8%,31.6%,4.5%,24.9%
Rural Areas,1.0%,47.1%,45.2%,0.3%,6.4%
Urban Unincorporated Areas,1.3%,41.6%,46.4%,0.4%,10.3%


In [23]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"15,514","504,665","526,699","6,574","159,010"
Core Cities,"46,935","1,351,749","1,314,042","58,697","427,932"
High Capacity Transit Communities,"41,824","1,299,068","1,273,502","40,527","379,228"
Metropolitan Cities,"86,392","1,534,179","1,318,086","188,165","1,038,088"
Rural Areas,"16,044","788,986","757,595","5,188","107,974"
Urban Unincorporated Areas,"7,251","230,795","257,283","2,244","57,291"


In [24]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,0.7%,76.5%,17.2%,0.9%,4.8%
Core Cities,1.2%,74.6%,16.4%,2.0%,5.8%
High Capacity Transit Communities,1.0%,76.0%,16.4%,1.7%,5.0%
Metropolitan Cities,2.4%,64.1%,13.1%,3.8%,16.7%
Rural Areas,0.5%,78.6%,16.2%,1.0%,3.8%
Urban Unincorporated Areas,0.7%,77.7%,16.9%,0.9%,3.7%


In [25]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"1,159","134,512","30,194","1,511","8,434"
Core Cities,"6,117","366,071","80,281","9,688","28,404"
High Capacity Transit Communities,"4,419","342,206","73,651","7,476","22,440"
Metropolitan Cities,"17,158","463,713","94,595","27,303","120,884"
Rural Areas,"1,030","177,440","36,680","2,190","8,515"
Urban Unincorporated Areas,506,"58,749","12,799",707,"2,825"


In [26]:
equity_geogs = summary_config['hh_equity_geogs']

trip_equity_geog = pd.DataFrame()
df_all = pd.DataFrame()
df_work = pd.DataFrame()
df_non_work = pd.DataFrame()
trip_work = pd.DataFrame()
trip_non_work = pd.DataFrame()

for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    df['EFA'] = geog
    # trip_equity_geog = pd.concat([trip_equity_geog, df[['equity_geog_type', 'dpurp', 'Mode', 'equity_geog','trexpfac', 'Trip Type']]])

    _df_all, _df_work, _df_non_work, _trip_all, _trip_work, _trip_non_work = calc_mode_share(df, 'equity_geog')

    _df_all['EFA'] = geog
    _df_work['EFA'] = geog
    _df_non_work['EFA'] = geog
    _trip_all['EFA'] = geog
    _trip_work['EFA'] = geog
    _trip_non_work['EFA'] = geog


    df_all = pd.concat([df_all, _df_all])
    df_work = pd.concat([df_work, _df_work])
    df_non_work = pd.concat([df_non_work, _df_non_work])
    trip_all = pd.concat([trip_all, _trip_all])
    trip_work = pd.concat([trip_work, _trip_work])
    trip_non_work = pd.concat([trip_non_work, _trip_non_work])

col_dict = {'hh_efa_dis': 'Disability',
                                'hh_efa_pov200': 'Income',
                                'hh_efa_poc': 'People of Color',
                                'hh_efa_lep': 'LEP',
                                'hh_efa_older': 'Older Adult',
                                'hh_efa_youth': 'Youth'}
col_list = ['EFA','Drove Alone', 'Shared Ride', 'Transit','Walk', 'Bike']

df_all['EFA'] = df_all['EFA'].map(col_dict)
df_all = df_all[col_list]
df_work['EFA'] = df_work['EFA'].map(col_dict)
df_work = df_work[col_list]
df_non_work['EFA'] = df_non_work['EFA'].map(col_dict)
df_non_work = df_non_work[col_list]
trip_all['EFA'] = trip_all['EFA'].map(col_dict)
trip_all = trip_all[col_list]
trip_work['EFA'] = trip_work['EFA'].map(col_dict)
trip_work = trip_work[col_list]
trip_non_work['EFA'] = trip_non_work['EFA'].map(col_dict)
trip_non_work = trip_non_work[col_list]

In [27]:
df_all

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,45.4%,36.1%,2.1%,14.9%,1.5%
Below Regional Average,Disability,45.8%,36.6%,2.2%,13.9%,1.5%
Higher Share of Equity Population,Disability,43.7%,34.5%,2.2%,18.1%,1.6%
Above Regional Average,Older Adult,46.3%,36.8%,2.0%,13.6%,1.4%
Below Regional Average,Older Adult,44.4%,35.9%,2.4%,15.7%,1.7%
Higher Share of Equity Population,Older Adult,47.2%,35.6%,2.1%,13.8%,1.3%
Above Regional Average,LEP,45.6%,36.8%,2.1%,14.0%,1.5%
Below Regional Average,LEP,45.5%,35.5%,2.1%,15.4%,1.5%
Higher Share of Equity Population,LEP,44.6%,37.6%,2.6%,13.7%,1.6%


In [28]:
df_non_work

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,41.3%,39.2%,2.1%,15.8%,1.5%
Below Regional Average,Disability,41.5%,40.0%,2.2%,14.7%,1.5%
Higher Share of Equity Population,Disability,40.0%,37.2%,2.2%,19.0%,1.6%
Above Regional Average,Older Adult,42.3%,39.9%,1.9%,14.4%,1.4%
Below Regional Average,Older Adult,40.0%,39.2%,2.4%,16.7%,1.7%
Higher Share of Equity Population,Older Adult,43.6%,38.4%,2.0%,14.7%,1.3%
Above Regional Average,LEP,41.4%,40.2%,2.1%,14.8%,1.5%
Below Regional Average,LEP,41.5%,38.7%,2.1%,16.3%,1.5%
Higher Share of Equity Population,LEP,40.2%,40.8%,2.7%,14.7%,1.6%


In [29]:
df_work

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,72.2%,15.5%,2.2%,8.7%,1.4%
Below Regional Average,Disability,72.4%,15.4%,2.3%,8.5%,1.4%
Higher Share of Equity Population,Disability,70.1%,14.7%,2.3%,11.4%,1.6%
Above Regional Average,Older Adult,73.1%,15.6%,2.0%,8.1%,1.2%
Below Regional Average,Older Adult,71.1%,15.2%,2.3%,9.7%,1.6%
Higher Share of Equity Population,Older Adult,73.5%,15.0%,2.8%,7.5%,1.2%
Above Regional Average,LEP,72.2%,15.6%,1.9%,9.0%,1.3%
Below Regional Average,LEP,71.7%,14.9%,2.5%,9.5%,1.4%
Higher Share of Equity Population,LEP,72.8%,16.6%,2.2%,7.0%,1.5%


## Mode Share by Trip Destination

In [30]:
tour_rgc_dest = pd.read_csv(output_path / 'agg/dash/tour_rgc_dest.csv')
trip_rgc_dest = pd.read_csv(output_path / 'agg/dash/trip_rgc_dest.csv')

# order RGCs

# all individual centers
l = tour_rgc_dest['tour_d_rgc'].sort_values().unique().tolist()
l.remove('Not in RGC')
l.remove(np.nan)

# aggregated centers
geog_order = ["Region","In RGC","Not in RGC"]

# combine
geog_order.extend(l)

In [31]:
def get_rgc_location(df, dest_col):
    # regional data
    df_region = df.copy()
    df_region[dest_col] = "Region"

    # aggregated centers
    df_isrgc = df.copy()
    df_isrgc.loc[df_isrgc[dest_col]!="Not in RGC", dest_col] = "In RGC"


    df_rgc = pd.concat([
        df_region,
        df_isrgc,
    # 2025 notes:
    # 7 records in the data are missing both mode and and RGC values
    # remove these records
        df.loc[~df[dest_col].isna()]
        ])

    df_rgc[dest_col] = pd.Categorical(df_rgc[dest_col], ordered=True,
                    categories=geog_order)
    
    return df_rgc

df_tour = get_rgc_location(tour_rgc_dest, 'tour_d_rgc')
df_trip = get_rgc_location(trip_rgc_dest, 'trip_d_rgc')




In [32]:
def calc_mode_share_by_rgc(df_trip_geog, geog_col, mode_col, n_trip_col):

    df = df_trip_geog.pivot_table(columns=mode_col, index=geog_col, values=n_trip_col, aggfunc='sum', observed=True)

    df = df.apply(lambda x: x/ df_trip_geog.groupby(geog_col, observed=True)[n_trip_col].sum())

    return df

\[TRIP\] Mode Share for All Purposes by Regional Center (Destination Location)

In [33]:
calc_mode_share_by_rgc(df_trip, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,21.3%,14.1%,44.5%,1.6%,0.3%,2.1%,14.5%
In RGC,1.9%,15.9%,8.9%,42.0%,0.5%,0.8%,5.5%,24.6%
Not in RGC,1.4%,22.4%,15.1%,45.0%,1.8%,0.2%,1.5%,12.5%
Auburn,1.1%,22.3%,12.4%,47.9%,0.5%,0.2%,3.2%,12.3%
Bellevue,1.7%,15.2%,8.8%,43.0%,0.4%,0.5%,2.5%,28.0%
Bothell Canyon Park,1.1%,22.1%,14.3%,54.1%,1.1%,0.2%,0.4%,6.7%
Bremerton,1.5%,15.5%,9.0%,54.6%,0.1%,0.3%,1.7%,17.3%
Burien,1.1%,23.2%,12.9%,45.4%,0.8%,0.2%,2.3%,13.9%
Everett,1.2%,18.8%,10.9%,47.6%,0.6%,0.3%,1.6%,19.0%


\[TRIP\] Mode Share to Work by Regional Center (Destination Location)

In [34]:
df_trip_work = df_trip[df_trip['dpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_trip_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.4%,9.4%,5.8%,71.6%,0.0%,0.6%,2.3%,8.9%
In RGC,1.8%,8.5%,5.3%,62.0%,0.0%,1.0%,4.9%,16.5%
Not in RGC,1.2%,9.8%,6.1%,76.6%,0.0%,0.4%,0.9%,5.0%
Auburn,1.0%,9.4%,6.7%,74.4%,NaN,0.4%,1.0%,7.1%
Bellevue,1.5%,9.1%,6.2%,65.1%,NaN,0.5%,1.7%,15.9%
Bothell Canyon Park,1.0%,9.7%,5.8%,79.5%,NaN,0.3%,0.2%,3.5%
Bremerton,1.1%,10.0%,6.3%,74.6%,NaN,0.3%,1.5%,6.2%
Burien,1.5%,9.8%,6.6%,73.5%,NaN,0.3%,1.4%,7.0%
Everett,1.3%,9.2%,5.4%,72.8%,0.0%,0.2%,0.6%,10.5%


\[TRIP\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [35]:
df_trip_non_work = df_trip[df_trip['dpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_trip_non_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,23.2%,15.4%,40.4%,1.8%,0.3%,2.1%,15.3%
In RGC,1.9%,18.7%,10.2%,34.4%,0.6%,0.7%,5.7%,27.7%
Not in RGC,1.5%,23.9%,16.2%,41.3%,2.0%,0.2%,1.6%,13.4%
Auburn,1.1%,24.7%,13.5%,43.0%,0.6%,0.2%,3.6%,13.3%
Bellevue,1.8%,18.1%,10.0%,32.4%,0.6%,0.4%,2.9%,33.8%
Bothell Canyon Park,1.1%,27.2%,17.7%,43.8%,1.5%,0.2%,0.5%,8.0%
Bremerton,1.8%,20.8%,11.5%,35.5%,0.2%,0.3%,2.0%,27.8%
Burien,1.1%,24.5%,13.5%,42.7%,0.9%,0.2%,2.4%,14.6%
Everett,1.2%,21.5%,12.4%,40.5%,0.8%,0.3%,1.9%,21.3%


## Mode Share by Tour Destination

\[TOUR\] Mode Share for All Purposes by Regional Center (Destination Location)

In [36]:
calc_mode_share_by_rgc(df_tour, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,,
Region,1.3%,22.1%,19.3%,0.1%,37.9%,2.6%,0.5%,3.2%,13.0%
In RGC,1.5%,18.4%,14.2%,0.5%,39.5%,0.7%,1.0%,7.8%,16.4%
Not in RGC,1.3%,23.2%,20.7%,0.1%,37.4%,3.1%,0.3%,1.9%,12.0%
Auburn,0.7%,25.3%,20.0%,0.1%,42.1%,0.3%,0.3%,2.4%,8.7%
Bellevue,1.3%,17.7%,14.4%,0.2%,43.0%,0.3%,0.7%,3.4%,19.0%
Bothell Canyon Park,0.9%,24.0%,21.9%,0.0%,45.1%,1.9%,0.3%,0.9%,4.9%
Bremerton,1.2%,20.8%,15.9%,0.4%,50.1%,0.1%,0.4%,0.8%,10.3%
Burien,0.7%,24.5%,18.8%,0.0%,38.8%,1.6%,0.3%,4.0%,11.2%
Everett,1.0%,22.5%,17.8%,0.1%,40.8%,1.0%,0.4%,2.5%,13.9%


\[TOUR\] Mode Share to Work by Regional Center (Destination Location)

In [37]:
df_tour_work = df_tour[df_tour['pdpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_tour_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.4%,15.3%,12.5%,0.5%,62.2%,1.1%,2.7%,4.3%
In RGC,1.8%,14.0%,11.3%,1.1%,55.2%,1.8%,6.0%,8.8%
Not in RGC,1.2%,16.1%,13.1%,0.2%,66.0%,0.6%,0.9%,1.8%
Auburn,1.1%,16.5%,14.0%,0.3%,63.1%,0.8%,1.6%,2.7%
Bellevue,1.5%,15.3%,13.4%,0.4%,58.9%,1.0%,2.0%,7.5%
Bothell Canyon Park,0.9%,15.6%,13.0%,0.1%,68.8%,0.6%,0.3%,0.7%
Bremerton,0.9%,17.0%,13.8%,0.6%,64.3%,0.5%,1.2%,1.7%
Burien,1.4%,16.0%,13.0%,0.1%,64.6%,0.6%,1.7%,2.6%
Everett,1.3%,16.0%,12.8%,0.2%,63.3%,0.7%,0.6%,5.0%


\[TOUR\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [38]:
df_tour_non_work = df_tour[df_tour['pdpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_tour_non_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.3%,24.7%,21.8%,28.6%,3.5%,0.2%,3.4%,16.4%
In RGC,1.3%,21.9%,16.4%,27.3%,1.2%,0.4%,9.2%,22.4%
Not in RGC,1.3%,25.3%,23.0%,28.8%,4.0%,0.2%,2.2%,15.1%
Auburn,0.6%,28.4%,22.1%,34.7%,0.4%,0.2%,2.7%,10.8%
Bellevue,1.1%,20.8%,15.8%,22.7%,0.7%,0.2%,5.1%,33.7%
Bothell Canyon Park,0.8%,29.4%,27.8%,29.7%,3.1%,0.2%,1.3%,7.7%
Bremerton,1.7%,27.0%,19.2%,27.0%,0.3%,0.3%,0.2%,24.3%
Burien,0.6%,25.9%,19.8%,34.6%,1.9%,0.2%,4.3%,12.6%
Everett,0.8%,25.4%,20.1%,30.8%,1.5%,0.3%,3.3%,17.8%
